<a href="https://colab.research.google.com/github/emrapport/w266-final-project/blob/master/modeling/LP_CNN_OverSamp_ROCAUC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# this notebook is based off of this blog post: 
# https://realpython.com/python-keras-text-classification/#reader-comments

In [0]:
DATASET_NAME = "oversample"
MODEL_NAME = "oversample"
MAX_SEQ_LENGTH = 20
TRAINING_SET_SIZE = 4000000
VAL_SET_SIZE = 2000000
NUM_EPOCHS = 20
BATCH_SIZE = 1000
MAX_SEQUENCE_LENGTH = 20


# I don't think we have a good reason to do this right now
# but we might eventually 
SAVE_TOKENIZED_DATA = False

In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import HashingVectorizer
from scipy.sparse import hstack, vstack
from keras.preprocessing.text import Tokenizer
from tensorflow.keras import Sequential, layers
from keras.utils import plot_model
import pandas as pd
import numpy as np
import copy
import time
import pickle
!pip install gcsfs

pd.set_option('max_colwidth', 100)

Using TensorFlow backend.


     |████████████████████████████████| 51kB 1.3MB/s 
  Created wheel for gcsfs: filename=gcsfs-0.3.1-py2.py3-none-any.whl size=17936 sha256=e5134d9b11ff6d60703f6a438e83ef22094299a5f50b1d823307454785bc6dff
  Stored in directory: /root/.cache/pip/wheels/9d/2b/6f/86954f0d8caa1173841e62bb780dc0f8693bd268e04a267682
Successfully built gcsfs


In [3]:
# this cell is only necessary if running in colab
project_id = 'w266-251323'
import uuid
bucket_name = 'fb-congressional-data/'
from google.colab import auth
auth.authenticate_user()
!gcloud config set project {project_id}

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Updated property [core/project].


In [4]:
train_df = pd.read_csv("gs://fb-congressional-data/contraction_expanded_data/train.csv", index_col=0)

/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [5]:
dev_df = pd.read_csv("gs://fb-congressional-data/contraction_expanded_data/dev.csv", index_col=0)

/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [6]:
num_responses_per_post = train_df.post_id.value_counts().reset_index()
num_responses_per_post.columns = ['post_id', 'num_responses']
too_big_posts = num_responses_per_post[num_responses_per_post.num_responses > 50]
posts_to_sample = too_big_posts.post_id.values
# this gets all the rows for posts we DON'T need to sample 
new_train_df = train_df[~train_df.post_id.isin(posts_to_sample)]
# this should be true
assert(len(too_big_posts) + new_train_df.post_id.nunique() == train_df.post_id.nunique())
too_big_post_rows = train_df[train_df.post_id.isin(posts_to_sample)]
sampled_rows = too_big_post_rows.groupby('post_id').apply(lambda x: x.sample(n=50)).reset_index(drop=True)
new_train_df = pd.concat([new_train_df, sampled_rows])
new_train_df.shape

(3962284, 8)

In [0]:
# shuffle the data
# be sure to do this before you extract X's and y's!!
train_df = new_train_df.sample(frac=1)
dev_df = dev_df.sample(frac=1)

In [8]:
train_df.op_gender.value_counts()

M    2900753
W    1061531
Name: op_gender, dtype: int64

In [0]:
male = train_df[train_df.op_gender=='M']
female = train_df[train_df.op_gender=='W']

In [0]:
from sklearn.utils import resample

female_upsampled = resample(female,
                          replace=True, # sample with replacement
                          n_samples=len(male), # match number in majority class
                          random_state=27) # reproducible results


In [0]:
upsampled_train = pd.concat([male,female_upsampled])

In [14]:
upsampled_train.op_gender.value_counts()

M    2900753
W    2900753
Name: op_gender, dtype: int64

In [0]:
y_train = upsampled_train.op_gender.values
y_dev = dev_df.op_gender.values

In [0]:
def turn_to_ints(li):
    final_list = []
    for gender in li:
        if gender=='M':
            final_list.append(1)
        else:
            final_list.append(0)
    return final_list
            
y_train = turn_to_ints(y_train)
y_dev = turn_to_ints(y_dev)

In [0]:
y_train = np.asarray(y_train)
y_dev = np.asarray(y_dev)

In [0]:
def get_text_list(init_list):
    sentences = []
    for sentence in init_list:
        if type(sentence) != str:
            sentences.append("")
        else:
            sentences.append(sentence)
    return sentences

new_sentences_train = get_text_list(upsampled_train.response_text.values)
new_sentences_test = get_text_list(dev_df.response_text.values)

In [33]:
time_start = time.time()

# this is the default list of filters + apostrophe
# added because we have dealt with common contractions, so other apostrophes should mostly be possessive 
tokenizer = Tokenizer(num_words=200000, filters='!"\'#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n')
tokenizer.fit_on_texts(new_sentences_train)

X_train = tokenizer.texts_to_sequences(new_sentences_train)
X_test = tokenizer.texts_to_sequences(new_sentences_test)

vocab_size = len(tokenizer.word_index) + 1

currentTime = time.gmtime(time.time() - time_start)

#Convert the gmtime struct to a string
timeStr = time.strftime("%M minutes, %S seconds", currentTime)

print("Tokenized in {}".format(timeStr))

Tokenized in 07 minutes, 47 seconds


In [0]:
from keras.preprocessing.sequence import pad_sequences

X_train = pad_sequences(X_train, padding='post', maxlen=MAX_SEQUENCE_LENGTH)
X_test = pad_sequences(X_test, padding='post', maxlen=MAX_SEQUENCE_LENGTH)

In [0]:
if SAVE_TOKENIZED_DATA:
  x_train_path = 'X_train_{}.pkl'.format(DATASET_NAME)
  x_dev_path = 'X_dev_{}.pkl'.format(DATASET_NAME)
  y_train_path = 'y_train_{}.pkl'.format(DATASET_NAME)
  y_dev_path = 'y_dev_{}.pkl'.format(DATASET_NAME)

  with open(x_train_path, 'wb') as file:
      pickle.dump(X_train, file)   
  with open(x_dev_path, 'wb') as file:
      pickle.dump(X_test, file)
  with open(y_train_path, 'wb') as file:
      pickle.dump(y_train, file)
  with open(y_dev_path, 'wb') as file:
      pickle.dump(y_dev, file)

  # copy to bucket
  !gsutil cp /content/{x_train_path} gs://fb-congressional-data/test
  !gsutil cp /content/{x_dev_path} gs://fb-congressional-data/
  !gsutil cp /content/{y_train_path} gs://fb-congressional-data/
  !gsutil cp /content/{y_dev_path} gs://fb-congressional-data/

In [38]:
!gsutil cp gs://fb-congressional-data/glove* /tmp/

Copying gs://fb-congressional-data/glove.6B.100d.txt...
Copying gs://fb-congressional-data/glove.6B.200d.txt...
Copying gs://fb-congressional-data/glove.6B.300d.txt...
Copying gs://fb-congressional-data/glove.6B.50d.txt...
- [4 files][  2.1 GiB/  2.1 GiB]   46.7 MiB/s                                   
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying gs://fb-congressional-data/glove.6B.zip...
\
Operation completed over 5 objects/2.9 GiB.                                      


In [0]:
def create_embedding_matrix(filepath, word_index, embedding_dim):
    vocab_size = len(word_index) + 1  # Adding again 1 because of reserved 0 index
    embedding_matrix = np.zeros((vocab_size, embedding_dim))

    with open(filepath) as f:
        for line in f:
            word, *vector = line.split()
            if word in word_index:
                idx = word_index[word] 
                embedding_matrix[idx] = np.array(
                    vector, dtype=np.float32)[:embedding_dim]

    return embedding_matrix

embedding_dim = 100
embedding_matrix = create_embedding_matrix(
                     '/tmp/glove.6B.{}d.txt'.format(embedding_dim),
                      tokenizer.word_index, embedding_dim)

In [0]:
# trying to figure out which words are empty here
# counter = 0
# empty_indexes = []
# for index, row in enumerate(embedding_matrix):
#   if sum(row) == 0:
#     empty_indexes.append(index)
#     counter += 1
#   if counter > 1000:
#     break

# for idx in empty_indexes:
#   try:
#     print(tokenizer.index_word[idx])
#   except:
#     print("No entry for {}".format(idx))


In [0]:
# hmmmm....
# nonzero_elements = np.count_nonzero(np.count_nonzero(embedding_matrix, axis=1))
# nonzero_elements / vocab_size

0.15381079935942088

In [40]:
embedding_matrix.shape

(590572, 100)

In [41]:
model = Sequential()
model.add(layers.Embedding(vocab_size, embedding_dim, 
                           weights=[embedding_matrix], 
                           input_length=MAX_SEQUENCE_LENGTH, 
                           trainable=False))
model.add(layers.Conv1D(128, 5, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 20, 100)           59057200  
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 16, 128)           64128     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1290      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 11        
Total params: 59,122,629
Trainable params: 65,429
Non-trainable params: 59,057,200
_________________________________________________________________


In [0]:
# smaller_X_train = X_train[:TRAINING_SET_SIZE]

In [0]:
# smaller_y_train = y_train[:TRAINING_SET_SIZE]

In [0]:
# # Set up oversampling with SMOTE (Synthetic Minority Oversampling Technique)
# from imblearn.over_sampling import SMOTE
# sm = SMOTE(random_state=12, ratio = 1.0) # Will bring final ratio to 1.0
# x_train_res, y_train_res = sm.fit_sample(X_train, y_train)


/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


In [0]:
smaller_X_dev = X_test[:VAL_SET_SIZE]
smaller_y_dev = y_dev[:VAL_SET_SIZE]

In [43]:
try:
  time_start = time.time()

  history = model.fit(X_train, y_train,
                      epochs=NUM_EPOCHS,
                      verbose=True,
                      validation_data=(smaller_X_dev, smaller_y_dev),
                      batch_size=BATCH_SIZE)

  currentTime = time.gmtime(time.time() - time_start)

  #Convert the gmtime struct to a string
  timeStr = time.strftime("%M minutes, %S seconds", currentTime)

  print("Trained in {}".format(timeStr))

except:
  currentTime = time.gmtime(time.time() - time_start)

  #Convert the gmtime struct to a string
  timeStr = time.strftime("%M minutes, %S seconds", currentTime)

  print("Trained in {}".format(timeStr))  

Train on 5801506 samples, validate on 2000000 samples
Epoch 1/20
5801506/5801506 [==============================] - 178s 31us/sample - loss: 0.6099 - acc: 0.6288 - val_loss: 0.6775 - val_acc: 0.5831
Epoch 2/20
5801506/5801506 [==============================] - 190s 33us/sample - loss: 0.5914 - acc: 0.6450 - val_loss: 0.6826 - val_acc: 0.5637
Epoch 3/20
5801506/5801506 [==============================] - 190s 33us/sample - loss: 0.5847 - acc: 0.6512 - val_loss: 0.6273 - val_acc: 0.6452
Epoch 4/20
5801506/5801506 [==============================] - 195s 34us/sample - loss: 0.5805 - acc: 0.6555 - val_loss: 0.6346 - val_acc: 0.6353
Epoch 5/20
5801506/5801506 [==============================] - 192s 33us/sample - loss: 0.5775 - acc: 0.6583 - val_loss: 0.6320 - val_acc: 0.6447
Epoch 6/20
5801506/5801506 [==============================] - 191s 33us/sample - loss: 0.5752 - acc: 0.6603 - val_loss: 0.6305 - val_acc: 0.6390
Epoch 7/20
5801506/5801506 [==============================] - 184s 32us/samp

In [46]:
loss, accuracy = model.evaluate(X_train, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_test, y_dev, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))


Training Accuracy: 0.6743
Testing Accuracy:  0.5984


The same model without oversampling went up 6 percentage points from 78% male baseline to 84% testing accuracy. The oversampled model started at 50/50 (50% baseline) and went up almost 10 percentage points to 59.8%.

In [47]:
# ROC AUC
from sklearn.metrics import roc_auc_score
# predict probabilities for test set
yhat_probs = model.predict(X_test, verbose=0)
# reduce to 1d array
yhat_probs = yhat_probs[:,0]
auc = roc_auc_score(y_dev, yhat_probs)
print('ROC AUC:', auc)


ROC AUC: 0.6150410537798867


In [0]:
# can't get this to work
plot_model(history)

In [0]:
preds = model.predict(X_test)

In [0]:
dev_df['probs'] = preds

In [0]:
def pred_to_label(row):
  if row['probs'] >= .5:
    return 'M'
  else:
    return 'W'

dev_df['preds'] = dev_df.apply(pred_to_label, axis=1)

In [0]:
wrong_preds = dev_df[dev_df.op_gender!=dev_df.preds]

In [52]:
wrong_preds.op_gender.value_counts()

M    767630
W    153133
Name: op_gender, dtype: int64

In [53]:
# total proportion of preds for women
dev_df.preds.value_counts()['W'] / len(dev_df)

0.41826074934569957

In [54]:
#random sample of mistakes
wrong_preds.head(20)

,original_idx,op_id,op_gender,post_id,responder_id,response_text,op_name,op_category,probs,preds
2284830,13468292,36181883,M,517694,Rik,it was time to stop obama care before it became a law. not 72 hours before it goes into effect.,Steve Scalise,Congress_Republican,0.463311,W
1903877,10936805,10458808,M,386642,Christine,and let us defend women's independence form forced reproduction!,Mark Takano,Congress_Democratic,0.351873,W
1162442,6608626,48816171,M,252204,Carol,she has beautiful! congrats to all!,Kenny Marchant,Congress_Republican,0.340864,W
2124889,11265869,77234050,W,399735,Marilyn,here's another article that lists where the info comes from. http://www.sensibleregulations.org...,Cathy McMorris Rodgers,Congress_Republican,0.873996,M
485376,4067575,77914417,M,140830,Mikhaila,thank you,John Garamendi,Congress_Democratic,0.435204,W
48345,2166260,38794897,M,36849,Pat,got my vote sherrod! keep on keeping on...we will make the place better!!,Sherrod Brown,Congress_Democratic,0.169555,W
1421,2119336,38794897,M,36268,Frank,dear sherrod brown: i see you are up for re-election in 2018. i could tell because you seem to ...,Sherrod Brown,Congress_Democratic,0.494041,W
593833,4842887,46506154,M,160190,Debi,beautiful family.,Marco Rubio,Congress_Republican,0.446439,W
788780,5037834,46506154,M,160609,Mònica,i loved everything you had to say and your presentation was great! you have my support! god bles...,Marco Rubio,Congress_Republican,0.483499,W
563029,4812083,46506154,M,160141,John,only until nov 2016.,Marco Rubio,Congress_Republican,0.452823,W


In [55]:
# most confident wrong predictions where the answer was F but they thought M
# same as before - references to a male politician 
wrong_preds.sort_values('probs', ascending=False)[:20]

,original_idx,op_id,op_gender,post_id,responder_id,response_text,op_name,op_category,probs,preds
1930513,10979905,86569077,W,389357,Jeremy,"thank you senator gillibrand. just called my senator, durbin, to ask him to support your filibus...",Kirsten Gillibrand,Congress_Democratic,1.000000,M
2001084,11050476,86569077,W,390043,Margaret,"the shooter took his mother's gun. if these guns were still banned, she would not have it. mr ra...",Kirsten Gillibrand,Congress_Democratic,1.000000,M
1993131,11042523,86569077,W,389977,Colleen,you mean mr paul mark? rand opportunist paul?,Kirsten Gillibrand,Congress_Democratic,0.999999,M
2288789,13624454,4938109,W,528242,AnneMarie,i am writing you as a concerned voter and animal lover regarding house resolution 752 – condemni...,Yvette Clarke,Congress_Democratic,0.999999,M
1930430,10979822,86569077,W,389355,Andrea,"this is beyond unacceptable. I had like to know what citizens can do, and what elected officials...",Kirsten Gillibrand,Congress_Democratic,0.999995,M
2052572,11101964,86569077,W,390749,Peter,"damn, even jim inhofe is upset with bunning.",Kirsten Gillibrand,Congress_Democratic,0.999993,M
1992461,11041853,86569077,W,389973,Jeannie,http://www.thedailybeast.com/articles/2013/07/02/kirsten-powers-i-don-t-stand-with-wendy-davis.html,Kirsten Gillibrand,Congress_Democratic,0.999992,M
2038475,11087867,86569077,W,390492,Leah,"i hate to be crass, but go fuck yourself al. ~signed an elitist feminist.",Kirsten Gillibrand,Congress_Democratic,0.999991,M
1990692,11040084,86569077,W,389945,Mark,the people of colorado have spoken. http://abcnews.go.com/us/wirestory/colo-lawmakers-face-rec...,Kirsten Gillibrand,Congress_Democratic,0.999991,M
2000968,11050360,86569077,W,390043,Matt,stand with rand or be like kirsten and give up our countries gun rights to the united nations.,Kirsten Gillibrand,Congress_Democratic,0.999990,M


In [56]:
# most confident wrong predictions where the answer was F but they thought M
# same as before - references to a male politician 
wrong_preds.sort_values('probs', ascending=True)[:20]

,original_idx,op_id,op_gender,post_id,responder_id,response_text,op_name,op_category,probs,preds
1065864,5314918,46506154,M,161457,Susan,love mia - pulling for her to have a huge win in utah - we need you in congress girl.,Marco Rubio,Congress_Republican,0.000012,W
1636803,10117662,51495713,M,370156,Eva,us representative mia love's speech was outstanding. we are loving us rep. mia love. ~ l (al...,Richard Shelby,Congress_Republican,0.000033,W
181521,2358273,62246450,M,52502,Annie,keep going elizabeth! we support you!,Ron Wyden,Congress_Democratic,0.000045,W
1516442,7934736,99469231,M,296388,Fred,"the white house,via rice,knowingly lied to the citizens of america and the united nations. we a...",John McCain,Congress_Republican,0.000050,W
232018,2408770,62246450,M,52977,Sean,"i agree with lanaya. i am happy to say I am an oregonian now, but i was born and raised in misso...",Ron Wyden,Congress_Democratic,0.000053,W
1065840,5314894,46506154,M,161457,Manda,<3 mia love!,Marco Rubio,Congress_Republican,0.000062,W
2180513,11321493,95515852,M,402017,Muha,love u jan,Bill Huizenga,Congress_Republican,0.000066,W
1238010,7364578,8140646,M,276480,Karen,wrote congresswoman granger who is my rep. hope it helps! thanks for all you are doing marc.,Marc Veasey,Congress_Democratic,0.000073,W
2252355,13071222,33944100,M,481577,Lisa,"we would have liked to of heard your questions for hillary robert, we wish you could have been o...",Robert Aderholt,Congress_Republican,0.000088,W
253237,2508436,23785853,M,62379,Dustin,"""his counterpart in the united states senate?"" you mean democrat sen. patty murray, right?",Mike Simpson,Congress_Republican,0.000090,W
